# Import

In [1]:
import pandas as pd
import numpy as np
import requests
import json

# Parameters

In [2]:
api_key = 'token'
url = f'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?api_key={api_key}&fuel_type=ELEC'

output_filename = 'AFDC EV Station.csv'

# API Call

In [3]:
r = requests.get(url)
data = r.json()

In [4]:
data['fuel_stations'][0]

{'access_code': 'private',
 'access_days_time': 'Fleet use only',
 'access_detail_code': None,
 'cards_accepted': None,
 'date_last_confirmed': '2023-01-10',
 'expected_date': None,
 'fuel_type_code': 'ELEC',
 'groups_with_access_code': 'Private',
 'id': 1517,
 'open_date': '1999-10-15',
 'owner_type_code': 'LG',
 'status_code': 'E',
 'restricted_access': None,
 'station_name': 'LADWP - Truesdale Center',
 'station_phone': None,
 'updated_at': '2023-02-15T22:45:41Z',
 'facility_type': 'UTILITY',
 'geocode_status': 'GPS',
 'latitude': 34.2483191527193,
 'longitude': -118.3879713743439,
 'city': 'Sun Valley',
 'intersection_directions': None,
 'plus4': None,
 'state': 'CA',
 'street_address': '11797 Truesdale St',
 'zip': '91352',
 'country': 'US',
 'bd_blends': None,
 'cng_dispenser_num': None,
 'cng_fill_type_code': None,
 'cng_psi': None,
 'cng_renewable_source': None,
 'cng_total_compression': None,
 'cng_total_storage': None,
 'cng_vehicle_class': None,
 'e85_blender_pump': None,
 '

# Transformation

In [5]:
# Create containers for dataframe
station_id_list = []
station_name_list = []
ev_netowrk_list = []
ev_connection_list = []
address_list = []
state_list = []
city_list = []
access_list = []
station_status_list = []
open_date_list = []
lat_list = []
lng_list = []
facility_list = []
price_list = []

In [6]:
# Transformation
# Go through each records provided from AFDC api
# Keep only records that are necessary
# Clean then store within containers

for i in range(len(data['fuel_stations'])):
  # station id
  id = data['fuel_stations'][i]['id']
  station_id_list.append(id)

  # station name
  station_name = data['fuel_stations'][i]['station_name']
  station_name_list.append(station_name)

  # ev netowrk
  ev_network = data['fuel_stations'][i]['ev_network']
  ev_netowrk_list.append(ev_network)

  # ev connection
  ev_connection_raw = str(data['fuel_stations'][i]['ev_connector_types'])

  # Clean the output
  ev_connection_step1 = ev_connection_raw.replace('[', '')
  ev_connection_step2 = ev_connection_step1.replace(']', '')
  ev_connection = ev_connection_step2.replace("'", "")

  ev_connection_list.append(ev_connection)

  # address
  street = data['fuel_stations'][i]['street_address']
  city = data['fuel_stations'][i]['city']
  state = data['fuel_stations'][i]['state']
  zip = data['fuel_stations'][i]['zip']

  address = f'{street}, {city}, {state}, {zip}'
  address_list.append(address)

  state_list.append(state)

  city_list.append(city)

  # station status
  station_status_initial = data['fuel_stations'][i]['status_code']

  if station_status_initial.upper() == 'P':
    station_status = False
  else:
    station_status = True

  station_status_list.append(station_status)

  # public access
  access_raw = data['fuel_stations'][i]['groups_with_access_code']

  # Clean output
  if 'PUBLIC' in access_raw.upper():
    access = True
  else:
    access = False

  access_list.append(access)

  # open date
  if station_status_initial.upper() == 'P':
    open_date = data['fuel_stations'][i]['expected_date']
  else:
    open_date = data['fuel_stations'][i]['open_date']

  open_date_list.append(open_date)

  # lat and lng
  lat = data['fuel_stations'][i]['latitude']
  lng = data['fuel_stations'][i]['longitude']

  lat_list.append(lat)
  lng_list.append(lng)

  # facility type
  facility = data['fuel_stations'][i]['facility_type']
  facility_list.append(facility)

  # ev pricing
  price = data['fuel_stations'][i]['ev_pricing']
  price_list.append(price)


In [7]:
# Create Dataframe
df = pd.DataFrame({'station_id': station_id_list,
                   'station_name': station_name_list,
                   'address': address_list,
                   'state': state_list,
                   'city': city_list,
                   'ev_network': ev_netowrk_list,
                   'ev_connection': ev_connection_list,
                   'public_access': access_list,
                   'station_status': station_status_list,
                   'open_date': open_date_list,
                   'latitude': lat_list,
                   'longitude': lng_list,
                   'facility': facility_list,
                   'price': price_list})

In [8]:
df.head()

,station_id,station_name,address,state,city,ev_network,ev_connection,public_access,station_status,open_date,latitude,longitude,facility,price
0,1517,LADWP - Truesdale Center,"11797 Truesdale St, Sun Valley, CA, 91352",CA,Sun Valley,Non-Networked,"CHADEMO, J1772, J1772COMBO",False,True,1999-10-15,34.248319,-118.387971,UTILITY,None
1,1519,LADWP - West LA District Office,"1394 S Sepulveda Blvd, Los Angeles, CA, 90024",CA,Los Angeles,Non-Networked,J1772,False,True,2020-02-28,34.052542,-118.448504,UTILITY,Free
2,1523,Los Angeles Convention Center,"1201 S Figueroa St, Los Angeles, CA, 90015",CA,Los Angeles,Non-Networked,J1772,True,True,1995-08-30,34.040539,-118.271387,PARKING_GARAGE,Free; parking fee
3,1525,LADWP - John Ferraro Building,"111 N Hope St, Los Angeles, CA, 90012",CA,Los Angeles,Non-Networked,"CHADEMO, J1772, J1772COMBO",False,True,1999-10-15,34.059133,-118.248589,UTILITY,None
4,1531,LADWP - Haynes Power Plant,"6801 E 2nd St, Long Beach, CA, 90803",CA,Long Beach,Non-Networked,"CHADEMO, J1772, J1772COMBO",False,True,2018-05-01,33.759802,-118.096665,UTILITY,None


# Export file as csv

In [9]:
df.to_csv(output_filename, index=False)